# Dnamic beta


In [ ]:
import pandas as pd
from src.config import config

In [ ]:
# Beta calculation (use intraday?)
spy = pd.read_parquet(path=f"{config.data.iqfeed.daily.cleaned}/SPY_daily.parquet")
prices = pd.read_parquet(path=f"{config.data.iqfeed.daily.cleaned}/AAPL_daily.parquet")
X = pd.merge(prices, spy, left_index=True, right_index=True, suffixes=("_stock", "_SPY"))

In [ ]:
X["r_stock"] = X["adj_close_stock"]/X["adj_close_stock"].shift() - 1 
X["r_SPY"] = X["adj_close_SPY"]/X["adj_close_SPY"].shift() - 1 
X.dropna(inplace=True)
returns = X[["r_stock", "r_SPY"]]

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression(fit_intercept=False)
X = returns["r_SPY"].to_numpy().reshape(-1, 1)
Y = returns["r_stock"].to_numpy()
lr.fit(X, Y)

In [ ]:
print(f"{lr.intercept_=:.4f} \n{lr.coef_[0]=:.4f}")

In [ ]:
resid_variance = ((Y - lr.predict(X))**2).mean()
resid_variance

In [ ]:
returns = returns * 1000

In [ ]:
from arch import arch_model
# define lists for storing objects
coeffs = []
cond_vol = []
std_resids = []
models = []

In [ ]:
for asset in returns.columns:
    model = (arch_model(returns[asset], 
                        mean = 'Constant', 
                        vol = 'GARCH', 
                        p = 1, o = 0, q = 1)
             .fit(update_freq = 0, disp = 'off'))
    
    coeffs.append(model.params)
    cond_vol.append(model.conditional_volatility)
    std_resids.append(model.resid / model.conditional_volatility)
    models.append(model)

In [ ]:
# store the results in df
coeffs_df = pd.DataFrame(coeffs, index=returns.columns)
cond_vol_df = pd.DataFrame(cond_vol).transpose().set_axis(returns.columns, axis = 'columns')
std_resids_df = pd.DataFrame(std_resids).transpose().set_axis(returns.columns, axis = 'columns')

In [ ]:
# calculate the constant conditional correlation matrix (CCC) R:
R = std_resids_df.transpose().dot(std_resids_df).div(len(std_resids_df))
R

In [ ]:
dynamic_betas = cond_vol_df.r_stock * R.iloc[0, 1] / cond_vol_df.r_SPY
dynamic_betas

In [ ]:
resids = returns.r_stock - (returns.r_SPY) * dynamic_betas
(resids**2).mean()

In [ ]:
# calculate one step ahead forecastof the conditional covariance matrix
import numpy as np
diag = []
D = np.zeros((2, 2))

for model in models:
    diag.append(model.forecast(horizon = 1).variance.values[-1][0])
    
diag = np.sqrt(np.array(diag))
np.fill_diagonal(D, diag)

H = np.matmul(np.matmul(D, R.values), D)

## Factors

In [ ]:
factors = pd.read_csv("D:/data/F-F_Research_Data_Factors_daily.CSV")
factors.columns = ["date", "Mkt-RF", "SMB", "HML", "RF"]
factors["date"] = pd.to_datetime(factors.iloc[:, 0], format="%Y%m%d")
factors.set_index("date", inplace=True)

In [ ]:
A = pd.merge(X.r_stock*100, factors, left_index=True, right_index=True, how="inner").dropna()


In [ ]:
A.iloc[0, :] = 0
(1 + A/100).apply(np.cumprod, axis=0).plot(logy=True)

In [ ]:
factors.rolling(252).corr(X.r_stock)